# Figuring out the relevant data for each research question

The first and arguably most important part of the data preprocessing in this case, is to figure out how to use the functionnalities already set-up in the **PatentsView API** to their fullest extent. This first part, below, focuses on the results we obtained after going through the documentation that was provided with the API, as well as from the preliminary data experimentation.

The most relevant documentation can be found here:
- [Query Language Documentation](http://www.patentsview.org/api/query-language.html)
- [Field List](http://www.patentsview.org/api/patent.html)

For each research question, we came up with a list of required output **fields** which the API calls need to return, as well as a list of input **filters** that limit the amount of extra pre-processing we need to do, while at the same time providing us with all the information necessary for us to answer throughly the research questions.

**N.B. the ordering of the questions is reversed, as after the preprocessing of the data, we realized reversing the question order seemed more logical for the data story.**

## Q2/ How can we best identify and visualize different geographical innovation networks? Can we estimate the number of people in such networks?

For this first part, we need the following output **fields**:
- `patent_number`: US Patent number, as assigned by USPTO.
- `assignee_latitude`: Latitude for assignee's location as listed on the patent.
- `assignee_longitude`: Longitude for assignee's location as listed on the patent.
- `cited_patent_number`: Patent number of cited patent.
- `inventor_latitude`: Latitude of inventor's as listed on the selected patent.
- `inventor_longitude`: Longitude of inventor's city as listed on the selected patent.
- `inventor_lastknown_latitude`: Latitude of inventor's city as of their most recent patent grant date.
- `inventor_lastknown_longitude`: Longitude of inventor's city as of their most recent patent grant date.
- `patent_type`: Category of patent (see below).

We chose to query for the latitudes and longitudes instead of for the location in the format city-state-country, as the former data are more useful for visualization purposes. Furthermore, during our preliminary data analysis, the data in the city-state-country format was not uniform (i.e., some cities were named in full, while others were abbreviated in different ways). On the other hand, the latitudes and longitudes data is uniform, and the missing data is also easier to clean than with the other format.

The reason for including the `patent_type` field is that we want to be able to distinguish between major patent categories:
- 'Defensive Publication': "... an intellectual property strategy used to prevent another party from obtaining a patent on a product, apparatus or method for instance." ( [wikipedia](https://en.wikipedia.org/wiki/Defensive_publication) )
- 'Design': "... legal protection granted to the ornamental design of a functional item" ( [wikipedia](https://en.wikipedia.org/wiki/Design_patent) )
- 'Plant': covering any "new variety of plant" ( [wikipedia](https://en.wikipedia.org/wiki/Plant_breeders'_rights) )
- 'Reissue': correction of "a significant error in an already issued patent" ( [uslegal](https://definitions.uslegal.com/r/reissue-patent/) )
- 'Statutory Invention Registration': "for publishing patent applications on which they no longer felt they could get patents" ( [wikipedia](https://en.wikipedia.org/wiki/United_States_Statutory_Invention_Registration) )
- 'Utility': patent for a "useful" patent ( [wikipedia](https://en.wikipedia.org/wiki/Utility_(patent)) )

We will also need to be able to query for patents based on the following **filters**:
- `patent_number`: US Patent number, as assigned by USPTO.
- `app_date`: Date the patent application was filed (filing date)

For the purposes of situating in time the patent data we have, we chose to consider the date at which the patent application was filed, instead of the date at which it was granted. The reasoning behind this choice is that at the time of the patent application, the innovation supporting it already exists. So while using the application date will most probably reduce the amount of most-recent data we can work with, in our opinion it will paint a more vivid picture of innovation. Furthermore, as our analysis is performed on data spanning a few decades, we're confident this choice will not be detremental to our story. 

### Q2.1/ If we then take a few examples of different types of companies and look at the network of patents supporting their own patents, will these networks match up with the former innovation networks, or will they be more self-contained? In the latter case, can we estimate the number of people that make up these networks? Are these innovation networks concentrated around specific areas, or are they spread out ?

This part does not require any extra fields than in the first case. On the other hand, it will be necessary for us to be able to query for the patents owned by specific companies. To this end, we need the following **filter**:
- `assignee_organization`: Organization name, if assignee is organization

The process we will follow in order to answer this question is as follows:
- Settle on a few examples of companies operating in the same industry (e.g., Facebook, Twitter, Snapchat or Apple, Samsung, Huawei).
- For each of the chosen companies, identify the patents they held a given point in time
- Identify the network of innovators behind these patents
- Visualize the networks for each of the companies
- Analyze the results - see Q.1.2

For this question, as well as the following ones, we will also try to design and implement an interactive way to visualize the results. 

### Q2.2/ Do similar companies use the same knowledge bases to innovate? For example, if we look at different social networking companies, will the networks supporting their patents be distinct? Will a given companies patents mostly cite their own previous patents, or will they tap outside innovation networks? On what scale? 

The features necessary to answer these questions have been covered above. 

### Q2.3/ What about if we look at university/academic knowledge bases and compare them with those of the companies analyzed above?

### Q2.4/ What about governmental or non-governmental organizations, or international agencies?

### Q2.5/ How have the innovation networks identified above evolved through time?

The process to be followed to answer these questions is similar to the process for **Q1.1**, with the exception that we need to be able to identify which patents belong to companies, organizations and governmental entities. To this end, we add the following output **fields** to our search queries:
- `assignee_type`: Classification of assignee.

The assignee classes are as follows:
- '2': US company or corporation
- '3': foreign company or corporation
- '4': US individual
- '5': foreign individual
- '6': US government
- '7': foreign government
- '8': country government
- '9': US state governement
- '1x': part interest

We will also need the following extra output **field**:
- `app_date`: Date the patent application was filed (filing date)

## Q1/ What does the typical patent-holder look like today (Corporation, Universities, Governments, Individuals), and how has that evolved throughout time / geographies?

### Q1.1/  Is a migration of innovators through time visible in the data, e.g. a convergence towards certain innovation centers?

### Q1.2 / How has the number of assignees and inventors evolved through for different patent types? Are there significant differences in these numbers between different geographies?

The features necessary to answer these questions have been covered above. 

# Gathering and pre-processing the data

The **pipeline.py** file contains the functions used to prepare the data for our analysis. The functions that are used at the pre-processing level are:
- The **patentsviewAPI()** function puts together the query string, the output fields string and the options string, and then extracts and saves the data returned by the PatentsView API in json format. The saved json data is of the following format (if there are no extra output fields added to the query):


    - page number in format '1', '2', ...
      - 'patents': list of patents in page. For each patent:
        - 'patent_number'
        - 'patent_type'
        - 'inventors' : list of inventors listed for the patent. For each inventor:
          - 'inventor_latitude'
          - 'inventor_longitude'
          - 'inventor_lastknown_latitude'
          - 'inventor_lastknown_longitude'
          - 'inventor_key_id'
        - 'assignees': list of assignees listed for the patent. For each assignee:
          - 'assignee_latitude'
          - 'assignee_longitude'
          - 'assignee_organization'
          - 'assignee_type'
          - 'assignee_key_id'
        - 'applications'
          - 'app_date'
          - 'app_id'
        - 'cited_patents' : list of other patents cited by the current patent. For each cited patent:
          - 'cited_patent_number'
      - 'count': number of results in the page
      - 'total_patent_count': total number of patents referenced in the results


- The **json_to_pandas()** function converts the saved json data from the PatentsView API to the format that is most convient for us for the pre-processing step - a set containing the following Pandas DataFrames:

  - `date`: associates a patent_number its application date.
  - `patent_type`: the number of patents in each patent_type category.
  - `assignee_type`: the number of patents in each assignee_type category.
  - `assignee_organization`: the number of patents belonging to each organization in the dataset.
  - `cited_patent_number`: the number of citations for each patent cited by a patent in the dataset.
  - `inventor_location`: inventor locations as stated in the patent application, indexed by patent_number.
  - `inventor_lastknown_location`: inventor locations as stated in the their most recent patent, indexed by patent number.
  - `assignee_location`: assignee locations as stated in the patent application, indexed by patent number.
  
  This function will make analyzing the 'raw' data more straightforward. Using this function to analyze the query results, we will then be able to specify the data cleaning process that is needed.
 
 
- The **clean_data()** function, which is detailed in the last section of this ipython notebook, converts the saved jsondata from the PatentsView API to the format that is most useful in answering the research questions - a set containing the following objects:

  - `assignees`: DataFrame with the following structure:
    - assignee_key_id (index)
    - assignee_type
    - assignee_organization
    - assignee_location, format (latitude, longitude)
    
  - `inventors`: DataFrame with the following structure:
    - inventor_key_id (index)
    - inventor_location, format (latitude, longitude)
    - inventor_last_location, format (latitude, longitude)
    
  - `patents`: Collection with the following structure:
    - patent_number (index)
    - patent_type
    - app_date
    - assignee_key_id, list containing all the patent assignees id (to match with `assignees`)
    - inventor_key_id, list containing all the patent inventors id (to match with `inventors`)
    
  - `citations`: DataFrame with the following structure:
    - cited_patent_number (index)
    - count


**As the maximum number of results per query is 10,000 per page, capped at 10 pages, in order to avoid hitting the limit, as a rule of thumb, if we are querying for all patent applications in a given date range, we need to keep the date range to about quarter of a year.**

# Example of analysis on a sample of data

In [1]:
import pandas as pd
from pipeline import patentsviewAPI, json_to_pandas

## Get and pre-process data

If the data has already been extracted from the PatentsView API, run second block, otherwise run first

In [2]:
datafile = patentsviewAPI(filename = '2015_q1', filepath = 'data',
                          app_date_from = '"2015-01-01"', app_date_to = '"2015-03-31"')
data = json_to_pandas(datafile)

fetching first page
fetching page 2
fetching page 3
fetching page 4
fetching page 5
fetching page 6
saving data


In [3]:
data = json_to_pandas(datafile)

## Analysis

In [4]:
data.keys()

dict_keys(['date', 'patent_type', 'assignee_type', 'assignee_organization', 'cited_patent_number', 'inventor_location', 'inventor_lastknown_location', 'assignee_location'])

Check the `date` dataframe. There should not be any inconsistencies here, as our query was based on the dates, so inconsistent dates would not have been returned.

In [5]:
data['date'].min().values[0], data['date'].max().values[0]

('2015-01-01', '2015-03-31')

In [6]:
sum(data['date'].values == None), sum(data['date'].values == float('nan'))

(array([0]), array([0]))

Check the `patent_type` dataframe.

In [7]:
data['patent_type']

,count
utility,47695
design,7218
plant,251
reissue,94


**Utility patents** are the most relevant for us. They are patents protecting "any new and useful process, machine, manufacture, or composition of matter, or any new and useful improvement thereof" ( [U.S. Code § 101](https://www.law.cornell.edu/uscode/text/35/101) ) These are the patents that we usually have in mind when we think of new technological innovations. On the other hand, **design** and **plant** patents are also relevant. What we see here is that these two categories are considerably smaller than the first. 

While we weren't necessarily expecting this result, it leads us to conclude that is it not necessary to discriminate between those three patent types. (If, e.g., design patent numbers had been more similary to utility patents, we could have chosen to only consider utility patents, or to split the analysis in two parts, as the two patent types refer to vastly different innovation types.) It will nonetheless be interesting to keep this difference in mind while answering the research questions.

As for the **reissue** patent type, we chose to disregard them, as they are not truly innovations, but only corrections on already issued patents. This category will be omitted directly when calling the API.

As for the two remaining patent types, **defensive publications** and **statutory invention registrations**, while they are not patents per say, they do refer to innovations, so they are relevant.

Check the `assignee_type` dataframe.

In [8]:
data['assignee_type']

,count
3,26723
2,25245
NaN,4444
4,224
5,177
6,160
7,38


In [9]:
data['assignee_type'].index.values == None

array([False, False,  True, False, False, False, False])

While the **NaNs** count in the `assignee_type` dataframe is negligeable as compared to the total count of assignees, there is an issue: assignee_types 4 and 5 refer to, respectively, US and foreign individuals. The total number of individuals assignees is thus much smaller than the NaN assignees count. Before removing the NaN values, we will need to investigate further what they may represent. 

Check the `assignee_organization` dataframe.

In [10]:
print(data['assignee_organization'].head(10)), print(data['assignee_organization'].tail(10))

                                                  count
NaN                                                4841
Samsung Electronics Co., Ltd.                      1550
International Business Machines Corporation        1161
Canon Kabushiki Kaisha                              696
LG Electronics Inc.                                 507
QUALCOMM Incorporated                               474
Kabushiki Kaisha Toshiba                            470
Seiko Epson Corporation                             470
Samsung Display Co., Ltd.                           419
Taiwan Semiconductor Manufacturing Company, Ltd.    385
                                              count
R.J. Reynolds Tobacco Products                    1
XI'AN AOLAN SCIENCE AND TECHNOLOGY CO., LTD.      1
Wowwee Group Limited                              1
Epoch Company, Ltd.                               1
VeraWall, LLC                                     1
Onepin, Inc.                                      1
Thirdwayv, Inc.     

(None, None)

In [11]:
data['assignee_organization'].index.values[0] == None

True

In [12]:
for assignee in data['assignee_organization'].index.values[1:]:
    if 'sony' in assignee.lower():
        print(assignee)

Sony Côrporation
Sony Mobile Communications Inc.
Sony Interactive Entertainment America LLC
Sony Semiconductor Solutions Corporation
SONY NETWORK ENTERTAINMENT INTERNATIONAL LLC
Sony Computer Entertainment Europe Limited
Sony Interactive Entertainment Europe Limited
Sony Computer Entertainment Inc.
SONY OLYMPUS MEDICAL SOLUTIONS INC.
Sony Corporation of America
Sony Europe Limited


The NaN count is refers should normally correspond to the total number of indiviuals assignee_type. In this case, though, we notice that it corresponds almost perfectly to the sum of the individual assignees count and NaN counts in the `assignee_type` dataframe.

While the organization names are definitely not in a standardized format, this will not be a problem, as we can simply use the list we obtain from querying for dates to get a list of the names used in the database.

In [13]:
sum(data['assignee_type']['count'].iloc[2:5]), sum(data['assignee_organization'].iloc[0])

(4845, 4841)

Check `cited_patent_number` dataframe.

In [14]:
print(data['cited_patent_number'].head(10)), print(data['cited_patent_number'].tail(10))

         count
NaN       6668
7674650    127
7732819    124
7297977    123
6294274    123
7282782    123
7385224    122
7064346    122
7061014    122
7323356    122
         count
6911243      1
6670521      1
6363530      1
440051       1
436738       1
419780       1
5034078      1
4640859      1
3894352      1
6309066      1


(None, None)

In [15]:
len(data['cited_patent_number']) - 1

611599

In [16]:
data['cited_patent_number'].describe()

,count
count,611600.000000
mean,1.712860
std,8.728264
min,1.000000
25%,1.000000
50%,1.000000
75%,2.000000
max,6668.000000


The NaN count here refers to the number of patent citations by the patents in our dataset which do not have a patent number. This is to say that the NaN count is an upper bound on the number of cited patents with no corresponding patent number. As the number of total cited patents is almost 100 times larger than the NaN count, we judge that we can safely disregard them and have them be skipped in the data cleaning process. Otherwise, we see that all entries are numeric, which suggests that the format is entirely uniform.

Check `inventor_location` dataframe.

In [17]:
data['inventor_location'].dtypes

lat    object
lon    object
dtype: object

The first thing to do will be to convert the latitudes and longitudes from objects to floats, while performing the data pre-processing.

In [18]:
data['inventor_location'].describe()

,lat,lon
count,148610,148610
unique,11977,12129
top,37.5665,126.978
freq,4425,4425


In [19]:
len(data['assignee_location'])

57011

In [20]:
sum(data['assignee_location'].values == None), sum(data['assignee_location'].values == float('nan'))

(array([4522, 4522]), array([0, 0]))

In [21]:
sum(data['assignee_location'].values == '0.1')

array([885, 885])

The major issue here is that almost 10% of the latitude and longitude data is either ('None','None') or ('0.1','0.1'). As our goal is to come up with a way to 'visualize' the dynamics of innovation, we need to discard this data. 

Check `inventor_location` and `inventor_lastknown_location` dataframes.

In [22]:
data['inventor_location'].dtypes, data['inventor_lastknown_location'].dtypes

(lat    object
 lon    object
 dtype: object, lat    object
 lon    object
 dtype: object)

In [23]:
len(data['inventor_location']), len(data['inventor_lastknown_location'])

(148633, 148633)

In [24]:
sum(data['inventor_location'].values == None), sum(data['inventor_location'].values == float('nan')), sum(data['inventor_location'].values == '0.1')

(array([23, 23]), array([0, 0]), array([1154, 1154]))

In [25]:
sum(data['inventor_lastknown_location'].values == None), sum(data['inventor_lastknown_location'].values == float('nan')), sum(data['inventor_lastknown_location'].values == '0.1')

(array([4, 4]), array([0, 0]), array([1110, 1110]))

Assuming that the missing values are non-overlapping between the `inventor_location` and `inventor_lastknown_location` dataframes, the upper bound on the proportion of missing data is here:

In [26]:
no_data = sum(data['inventor_location'].values == None) + sum(data['inventor_location'].values == '0.1')\
        + sum(data['inventor_lastknown_location'].values == None) + sum(data['inventor_lastknown_location'].values == '0.1')

round(no_data[0]/len(data['inventor_location']), 4)

0.0154

Coming back to the **NaN** count in the `assignee_type` dataframe, we look at the raw json data for these entries.

In [27]:
import json
import numpy as np

In [28]:
json_data = json.load(open(datafile))

In [29]:
missing_location_data = []
missing_assignee_type = {}

for page in json_data:
    for patent in json_data[page]['patents']:
        for inventor in patent['inventors']:
            if (inventor['inventor_latitude'] == None) or (inventor['inventor_latitude'] == '0.1'):
                missing_location_data.append(patent['patent_number'])
                
            if (inventor['inventor_lastknown_latitude'] == None) or (inventor['inventor_lastknown_latitude'] == '0.1'):
                missing_location_data.append(patent['patent_number'])

        for assignee in patent['assignees']:
            if (assignee['assignee_latitude'] == None) or (assignee['assignee_latitude'] == '0.1'):
                missing_location_data.append(patent['patent_number'])
                
            if (assignee['assignee_type'] == None):
                missing_assignee_type[patent['patent_number']] = assignee['assignee_organization']

We look at how many patent_numbers are missing at least one location datapoint.

In [30]:
len(pd.DataFrame(index = missing_location_data).index.unique())

6249

It is only a minority of the data points in this dataset which have only one location data missing. The majority of the data which have location data missing have both inventors and assignees location missing. 

In [31]:
len(missing_assignee_type)

4444

We now **merge** the `patent_number` data for the `missing_location_data` points and the `missing_assignee_type` points, and we count the number of unique `patent_number`. 

In [32]:
len(pd.DataFrame(index = np.vstack([np.array(list(missing_assignee_type.keys()))[:,None], np.array(missing_location_data)[:,None]])).index.unique())

6249

From the above code, we observe that it is only **a single** datapoint that has no `assignee_type` but **does** have `location` data. For us, this is good news. It means that by omitting the data which has no `assignee_type`, we do not risk losing more information than necessary - because in any case, we have to omit the data for which there is no location, as that is the most crucial information for our purposes. 

### Entire analysis
What is shown here is an example of the procedure that was followed to check the data. We went through it for random data samples spanning a few decades of data (one quarter year at a time). The results were all completely in line with the results shown above.

## Checking cleaned data

In [33]:
from pipeline import data_clean

In [34]:
data = data_clean(datafile)

In [35]:
data.keys()

dict_keys(['assignees', 'inventors', 'patents', 'citations'])

In [36]:
data['assignees'].head()

,type,organization,location
221223,2,MACNEIL IP LLC,"(41.6986, -88.0684)"
110443,3,Spanolux N.V.-Div. Balterio,"(50.9099, 3.3737)"
243235,3,PHIL AND TEDS MOST EXCELLENT BUGGY COMPANY LIM...,"(-36.8575, 174.81)"
90072,2,"Carestream Health, Inc.","(43.1566, -77.6088)"
351314,2,Intelius Inc.,"(33.7484, -117.875)"


In [37]:
data['inventors'].head()

,location,last_location
1291628,"(41.8009, -87.937)","(26.0571, -80.2507)"
2236915,"(42.1878, -88.183)","(42.1878, -88.183)"
2679416,"(51.0951, 5.7913)","(51.0951, 5.7913)"
2679417,"(50.7707, 3.8752)","(50.7707, 3.8752)"
1889088,"(40.8768, -73.3246)","(40.8768, -73.3246)"


In [38]:
data['patents'].head()

,assignees,date,inventors,type
8997419,[221223],2015-01-27,"[1291628, 2236915]",utility
8997430,[110443],2015-01-07,"[2679416, 2679417]",utility
9002934,[],2015-01-05,[1889088],utility
9004524,[243235],2015-01-15,"[2767897, 2767898]",utility
9005554,[],2015-01-12,[298359],utility


In [39]:
data['citations'].head()

,count
7674650,127
7732819,124
7297977,123
6294274,123
7282782,123
